In [1]:

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
plt.style.use(style='seaborn')
%matplotlib inline
import time

from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model #for lasso
from sklearn.model_selection import cross_val_predict
from sklearn import model_selection 


In [2]:
df = pd.read_csv('./Data/london_merged.csv')

In [3]:
df.sample(5)

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
5598,2015-08-25 21:00:00,834,18.0,18.0,78.0,24.0,2.0,0.0,0.0,1.0
5678,2015-08-29 05:00:00,126,16.0,16.0,80.0,6.5,1.0,0.0,1.0,1.0
8714,2016-01-03 23:00:00,140,8.0,6.5,87.0,9.0,1.0,0.0,1.0,3.0
12092,2016-05-24 17:00:00,3982,16.0,16.0,48.0,24.0,2.0,0.0,0.0,0.0
13195,2016-07-10 07:00:00,237,19.0,19.0,80.5,20.0,4.0,0.0,1.0,1.0


In [6]:
new_df = df.copy()
#adding 25% features to wind speed
new_df['wind_speed'] = new_df.apply(lambda row: row['wind_speed'] + np.random.random_sample()*10, axis=1 )
#adding 25% features to temp 
new_df['t2']  = new_df.apply(lambda row: row['t2'] + np.random.random_sample()*2, axis=1)

df = df.append(new_df.sample(int(new_df.shape[0]/4)))

#enhancing data
df['year'] = df['timestamp'].apply(lambda row: row[:4])
df['month'] = df['timestamp'].apply(lambda row: row[5:7])
df['hour'] = df['timestamp'].apply(lambda row: row[11:13])

df = df.drop(['timestamp'], axis =1)
df.sample(10)

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,year,month,hour
13587,2161,22.5,22.845463,48.5,22.720196,2.0,0.0,0.0,1.0,2016,07,15
12059,4640,13.5,13.500000,65.0,18.000000,1.0,0.0,0.0,0.0,2016,05,08
15017,106,13.0,14.776477,77.0,11.921506,1.0,0.0,0.0,2.0,2016,09,01
9372,260,7.0,5.000000,90.0,10.000000,7.0,0.0,1.0,3.0,2016,01,09
5111,1638,21.0,21.000000,50.0,16.000000,1.0,0.0,0.0,1.0,2015,08,14
11547,363,11.0,12.083467,77.0,19.223932,4.0,1.0,0.0,0.0,2016,05,00
14497,824,18.0,19.773970,68.0,32.419467,3.0,0.0,1.0,2.0,2016,09,09
2051,1053,11.5,13.223148,35.0,26.885107,1.0,0.0,0.0,0.0,2015,03,14
9984,105,0.0,-3.000000,83.5,9.000000,1.0,0.0,0.0,3.0,2016,02,05
6065,1386,16.0,17.350030,61.0,40.468309,2.0,0.0,0.0,2.0,2015,09,16


In [38]:
df['season'].unique()

array([3., 0., 1., 2.])

In [14]:
#generating data
def data_enhancement(data):
    
    gen_data = df.copy()
    for season in data['season'].unique():
        seasonal_data = gen_data[ gen_data['season'] == season]
        
        t1_std   = seasonal_data['t1'].std()
        t2_std   = seasonal_data['t2'].std()
        hum_std  = seasonal_data['hum'].std()
        wind_std = seasonal_data['wind_speed'].std()

        for i in range(seasonal_data.shape[0]):
            if np.random.randint(2) == 1: #losuje liczbe od 0 do 1  
                seasonal_data['t1'].values[i] += t1_std
            else:
                seasonal_data['t1'].values[i] -= t1_std   

            if np.random.randint(2) == 1:  
                seasonal_data['t2'].values[i] += t2_std
            else:
                seasonal_data['t2'].values[i] -= t2_std   
        
            if np.random.randint(2) == 1:   
                seasonal_data['hum'].values[i] += hum_std
            else:
                seasonal_data['hum'].values[i] -= hum_std

            if np.random.randint(2) == 1:  
                seasonal_data['wind_speed'].values[i] += wind_std
            else:
                seasonal_data['wind_speed'].values[i] -= wind_std
        
    return gen_data

gen = data_enhancement(df)                         


In [15]:
gen

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,year,month,hour
0,182,3.0,2.000000,93.0,6.000000,3.0,0.0,1.0,3.0,2015,01,00
1,138,3.0,2.500000,93.0,5.000000,1.0,0.0,1.0,3.0,2015,01,01
2,134,2.5,2.500000,96.5,0.000000,1.0,0.0,1.0,3.0,2015,01,02
3,72,2.0,2.000000,100.0,0.000000,1.0,0.0,1.0,3.0,2015,01,03
4,47,2.0,0.000000,93.0,6.500000,1.0,0.0,1.0,3.0,2015,01,04
...,...,...,...,...,...,...,...,...,...,...,...,...
15592,240,11.0,11.879080,77.0,19.896267,1.0,0.0,0.0,2.0,2016,10,00
12238,1442,17.0,17.692359,59.0,31.291870,2.0,1.0,0.0,0.0,2016,05,19
5180,2299,22.5,25.781207,59.0,25.559170,2.0,0.0,1.0,1.0,2015,08,11
13697,572,17.0,18.510319,66.0,17.847646,2.0,0.0,1.0,1.0,2016,07,08


In [17]:
cat_vars = ['weather_code', 'is_holiday','is_weekend','season','year','month']
num_var = ['t1','t2','hum','wind_speed']
num_var

['t1', 't2', 'hum', 'wind_speed']

In [28]:
y = df['cnt']
x = df.drop(['cnt'], axis=1)


In [36]:
x_train, x_val , y_train, y_val = model_selection.train_test_split(x,y ,test_size = 0.2 , random_state = 0)

#generating extar samples 

extra_sample = gen.sample(gen.shape[0] // 3 ) # generating 30% new data


In [37]:
extra_sample

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,year,month,hour
8029,100,13.0,13.000000,79.5,25.000000,7.0,0.0,1.0,3.0,2015,12,06
7451,1843,14.0,14.000000,77.0,18.000000,2.0,0.0,0.0,2.0,2015,11,19
9749,1731,2.5,0.893120,84.0,16.618204,1.0,0.0,0.0,3.0,2016,02,09
17219,231,5.0,2.609984,76.0,19.787003,1.0,1.0,0.0,3.0,2016,12,21
8642,595,7.0,4.500000,76.0,16.500000,1.0,0.0,0.0,3.0,2015,12,23
...,...,...,...,...,...,...,...,...,...,...,...,...
6180,2808,17.0,17.000000,56.0,7.000000,2.0,0.0,1.0,2.0,2015,09,12
2188,1496,10.0,9.667466,64.5,15.670036,3.0,0.0,1.0,0.0,2015,04,12
6451,2045,15.0,15.000000,65.5,19.000000,1.0,0.0,0.0,2.0,2015,09,19
4152,1372,23.0,23.000000,45.5,13.000000,1.0,0.0,0.0,1.0,2015,06,11


In [14]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

In [15]:
help(time.time())

Help on float object:

class float(object)
 |  float(x=0, /)
 |  
 |  Convert a string or number to a floating point number, if possible.
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __divmod__(self, value, /)
 |      Return divmod(self, value).
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __float__(self, /)
 |      float(self)
 |  
 |  __floordiv__(self, value, /)
 |      Return self//value.
 |  
 |  __format__(self, format_spec, /)
 |      Formats the float according to format_spec.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getnewargs__(self, /)
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __int__(self, /)
 |      int(self)
 |  
 |  __le__

In [16]:
from memory_profiler import profile

In [17]:


results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

# YOUR CODE HERE
#tree model
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size=0.2, 
    random_state=42,
)

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train) #fit transforms in fit for train data.
    total_time = time.time() - start_time
    
    pred = model.predict(x_val)  #transforms test data here (inside predict)
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
#results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')
results_ord

C:\Users\Wojtek\anaconda3\envs\dl\lib\site-packages\sklearn\metrics\_classification.py:1854: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


MemoryError: could not allocate 928776192 bytes